<h1><center>Laboratorio 9: Benchmark Estadístico con Reddit 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Gatica
- Nombre de alumno 2: Cristóbal Igor


### **Link de repositorio de GitHub:** `https://github.com/ngatica/LaboratoriosMDS`

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 3/12/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Utilizar la API de Reddit a partir de la librería `praw` y visualizar cuales post son más probables que sean puntuados positivamente.
- Utilizar `cache_lru` para cachear resultados de funciones.
- Aplicar un atajo estadístico para obtener la mean posterior de datos.
- Medir el tiempo de ejecución como también el uso de memoria de la función anterior.
- Optimizar la función anterior a través de `numba`.

#Importamos librerias utiles 😸

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%capture
!pip install praw
!pip install line_profiler
!pip install numba

%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://preview.redd.it/3sxusxcjge791.png?auto=webp&s=88bc1f9a3c59eafda24b0e32dd87d7cf596d205d">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema estadísticos, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis [1 Punto]

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a cada liena de código para ser ejecutada. Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del código a través de un perfilador.
- [ ] Estudiar la memoria ocupada por el código a través de un perfilador.
- [ ] Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?

In [3]:
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url


In [4]:
votes, post, url = praw_reddit()

### Estudiando los tiempos:

In [5]:
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-07 s

Total time: 18.4988 s
File: <ipython-input-3-bd47716803a4>
Function: praw_reddit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     2         2       5006.0   2503.0      0.0      reddit = praw.Reddit(
     3         1          8.0      8.0      0.0          client_id="-w2hyFINxZ8T3g",
     4         1          4.0      4.0      0.0          client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     5         1          4.0      4.0      0.0          password="ClasesMDS7202",
     6         1          4.0      4.0      0.0          user_agent="Clases",
     7         1          4.0      4.0      0.0          username="DocenciaDataScience",
     8         1          5.0      5.0      0.0          check_for_async=False,
     9                                               )
    10         1        178.0    178.0      0.0      subreddit = reddi

### Estudiando memoria

In [6]:
!pip install -q memory_profiler
%load_ext memory_profiler

In [7]:
%memit praw_reddit()

peak memory: 197.81 MiB, increment: 5.57 MiB


Creando modulo:

In [8]:
%%file praw_reddit_module.py
import praw
import numpy as np

def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
    subreddit = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit=n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions), 50)):
        top_n_submissions = top_submissions[:top_n]
        upvotes, downvotes, url[it], post[it] = [], [], [], []

        for submission in top_n_submissions:
            try:
                ratio = submission.upvote_ratio
                ups = int(
                    round((ratio * submission.score) / (2 * ratio - 1))
                    if ratio != 0.5
                    else round(submission.score / 2)
                )
                upvotes.append(ups)
                downvotes.append(ups - submission.score)
                post[it].append(submission.title)
                url[it].append(submission.url)
            except Exception as e:
                continue
        votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

Overwriting praw_reddit_module.py


In [9]:
from praw_reddit_module import praw_reddit
%mprun -f praw_reddit praw_reddit()

Filename: c:\Users\ngati\Downloads\praw_reddit_module.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     4    194.1 MiB    194.1 MiB           1   def praw_reddit(nombre_subreddit="chile", n_hot=1000):
     5    194.1 MiB      0.0 MiB           2       reddit = praw.Reddit(
     6    194.1 MiB      0.0 MiB           1           client_id="-w2hyFINxZ8T3g",
     7    194.1 MiB      0.0 MiB           1           client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
     8    194.1 MiB      0.0 MiB           1           password="ClasesMDS7202",
     9    194.1 MiB      0.0 MiB           1           user_agent="Clases",
    10    194.1 MiB      0.0 MiB           1           username="DocenciaDataScience",
    11    194.1 MiB      0.0 MiB           1           check_for_async=False,
    12                                             )
    13    194.1 MiB      0.0 MiB           1       subreddit = reddit.subreddit(nombre_subreddit)
    14                                        

**Respuesta:**

```Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.```

En cuanto a los tiempos de la función, el primer proceso que más tarda en la ejecución del código corresponde a la obtención y creación de la lista de los n_hot posts en el subreddit de Chile, lo cual puede ser mejorado si es que existe alguna forma de convertir a lista las submisiones de forma más eficiente (dependerá de la librería) o guardar esta misma en la memoria cache.

Luego, lo segundo que más tarda, es al momento de trabajar con las submisiones en un doble ciclo de for's, sobre todo al ejecutar el segundo for que calcula la cantidad de upvotes de cada submisión (la forma de calcular upvotes es lo que más tarda dada la utilización de if else). Es este doble for lo que se puede mejorar tratando de aplicar las operaciones de forma paralela para todos los valores al aprovechar las transformaciones que se pueden realizar a un array.

```Comente los resultados. ¿Qué lineas ocupan más memoria o más/menos tiempo?¿Por qué?```

En cuanto a la memoria, se puede observar un uso de memoria en torno a 230 MB, en dónde la mayor parte esta asociada a la ejecución completa de la función, teniendo sólamente un incremento de 9MB en la creación de la lista de los n_hot posts en el subreddit de Chile, que tambien corresponde a lo que toma más tiempo dentro de la función, por lo que el principal objetivo sería tratar esta línea de ejecución.

## 1.2 LRU y Análisis de Tiempo con Cache [1 Punto]

Respondan las siguientes preguntas:

1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder las preguntas.
- [ ] Mejorar el código con cache.
- [ ] Explicar las consecuencias de usar lru_cache sobre praw_reddit

**Respuestas Teóricas:**

`Escriba su respuesta aquí`
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
R: La memoria cache es una memoria que permite almacenar en el disco o en la RAM aquellos resultados de procesos previos con la finalidad de ser utilizados más adelante. Las siglas LRU hacen referencia a la oración en inglés Least Recently Used, y esto se refiere principalmente al orden de ejecución en el cual se almacenarán los procesos en la memoria cache.
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
R: Los costos que tiene la aplicación de técnicas de Caching vienen principalmente relacionadas al aumento en el uso de la memoria. Esto puede conllevar problemas debido a que la memoria cuenta con ciertas limitaciones dadas por las especificaciones de cada equipo.
3.	¿Cuál es la consecuencía de ocupar caching en la función anterior?.
R: Dentro de la función anterior el uso de memoria se encontraba cerca de los 230 MB y el uso de técnicas de caching aumentaría el uso en memoria.

In [19]:
###### Código Aquí ######
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [11]:
%%timeit
praw_reddit()

18.1 s ± 267 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
@lru_cache
def praw_reddit(nombre_subreddit="chile", n_hot=1000):
    reddit = praw.Reddit(
        client_id="-w2hyFINxZ8T3g",
        client_secret="zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw",
        password="ClasesMDS7202",
        user_agent="Clases",
        username="DocenciaDataScience",
        check_for_async=False,
    )
        
    subreddit  = reddit.subreddit(nombre_subreddit)

    votes, post, url = {}, {}, {}
    top_submissions = list(subreddit.hot(limit = n_hot))
    for it, top_n in enumerate(range(50, len(top_submissions),50)):
        top_n_submissions = top_submissions[:top_n]
        
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
    return votes, post, url

In [15]:
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-07 s

In [37]:
%%timeit
praw_reddit()

The slowest run took 6.50 times longer than the fastest. This could mean that an intermediate result is being cached.
443 ns ± 354 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Respuesta de lru_cache sobre praw_reddit**

`Como se ve, al utilizar LRU cache se baja el tiempo de ejecución de segundos a menos de un segundo,esto es debido a que este decorador permite memorizar las submisiones con las que se trabajan y que era lo que más tardaba en un inicio.`

## 1.3 Obtención de Mean Posterior y Standard Error [1 Punto]

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia estadística. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$

$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [39]:
###### Código Aquí ######
import math 
def intervalos(u, d):
    sum_u = 0
    sum_d = 0
    for i in range(len(u)):
        sum_u += u[i]
        sum_d += d[i]
    a = int(1+sum_u)
    b = int(1+sum_d)
    sigma = 1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma
    


def intervalos_numpy(u, d):
    sum_u = np.sum(u)
    sum_d = np.sum(d)
    a = int(1+sum_u)
    b = int(1+sum_d)
    sigma = 1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma

In [36]:
votes_in = votes[1]

In [31]:
%timeit intervalos(votes_in[:, 0], votes_in[:, 1])

30.2 µs ± 471 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
%timeit intervalos_numpy(votes_in[:, 0], votes_in[:, 1])

8.74 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [40]:
intervalos(votes_in[:, 0], votes_in[:, 1])

(0.8995487364620939, 0.004711774719470064)

In [41]:
intervalos_numpy(votes_in[:, 0], votes_in[:, 1])

(0.8995487364620939, 0.004711774719470064)

## 1.4 Comparación de Rendimiento [2 Puntos]

Ahora, aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. 

Para esto genere un gráfico de linea utilizando plotly, en donde se pueda observar el tiempo que toma ejecutar cada función (el eje y) diferentes cantidades de datos (ejemplo, en el eje x considerar 100, 200, 300, ..., 1000 datos) sobre cada función implementada (series `Python`, `Numpy`, `Python-JIT`, `Numpy-JIT`). ¿Es posible observar diferencias? ¿A qué se debe esto?.

**Nota:** Ejecuten las funciones compiladas con algún ejemplo antes de graficar. De lo contrario, les indicará el tiempo de compilación, cosa que no queremos medir

**Nota 2**: En el caso que su computador sea muy rápido, se recomienda graficar el eje y (tiempo) como logaritmo usando `log_y=True`.


In [42]:
###### Código Aquí ######
import math 
@jit(nopython=True)
def intervalos_JIT_py(u, d):
    sum_u = 0
    sum_d = 0
    for i in range(len(u)):
        sum_u += u[i]
        sum_d += d[i]
    a = int(1+sum_u)
    b = int(1+sum_d)
    sigma = 1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma
    
    
@jit(nopython=True)
def intervalos_numpy_JIT(u, d):
    sum_u = np.sum(u)
    sum_d = np.sum(d)
    a = int(1+sum_u)
    b = int(1+sum_d)
    sigma = 1.65 * math.sqrt(a*b/(((a+b)**2)*(a+b+1)))
    mu = a/(a+b)
    return mu,sigma

In [43]:
intervalos_JIT_py(votes_in[:, 0], votes_in[:, 1])

(0.8995487364620939, 0.004711774719470064)

In [44]:
intervalos_numpy_JIT(votes_in[:, 0], votes_in[:, 1])

(0.8995487364620939, 0.004711774719470064)

In [45]:
from timeit import timeit
tiempos_py = []
tiempos_numpy = []
tiempos_JIT_py = []
tiempos_numpy_JIT = []
for i in range(0, 18, 1):
    t_i = timeit(lambda: intervalos(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_py.append(t_i)
    t_i = timeit(lambda: intervalos_numpy(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_numpy.append(t_i)
    t_i = timeit(lambda: intervalos_JIT_py(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_JIT_py.append(t_i)
    t_i = timeit(lambda: intervalos_numpy_JIT(votes[i][:, 0], votes[i][:, 1]), number=1)
    tiempos_numpy_JIT.append(t_i)


In [46]:
print(tiempos_py)
print(tiempos_numpy)
print(tiempos_JIT_py)
print(tiempos_numpy_JIT)

[3.229999992981902e-05, 3.54000001152599e-05, 4.6300000121846097e-05, 0.00010919999999714491, 7.209999989754579e-05, 8.729999990464421e-05, 9.900000009110954e-05, 0.00011229999995521212, 0.00012440000000424334, 0.0001396999998632964, 0.0001536999998279498, 0.00017070000012608944, 0.00019079999992754892, 0.00020549999999275315, 0.00022209999997357954, 0.00022039999998924031, 0.00023569999984829337, 0.0002488000000084867]
[4.889999991064542e-05, 2.029999996011611e-05, 1.720000000204891e-05, 4.2200000052616815e-05, 1.9999999949504854e-05, 1.7900000102599734e-05, 1.629999997021514e-05, 1.5400000165755046e-05, 1.5699999948992627e-05, 1.619999989088683e-05, 2.0600000198101043e-05, 1.9300000076327706e-05, 2.6199999865639256e-05, 1.9099999917671084e-05, 1.889999998638814e-05, 1.8599999975776882e-05, 1.8299999965165625e-05, 1.7799999795897747e-05]
[7.599999889862374e-06, 3.400000196052133e-06, 3.1999998100218363e-06, 5.6000001222855644e-06, 3.5000000480067683e-06, 5.200000032345997e-06, 2.99999

In [47]:
x = list(range(50, 901, 50))

In [48]:
import pandas as pd
data = {'Datos' : x,
        'Tiempos_py': tiempos_py,
        'Tiempos_numpy': tiempos_numpy,
        'Tiempos_JIT_py': tiempos_JIT_py,
        'Tiempos_numpy_JIT': tiempos_numpy_JIT}
df = pd.DataFrame(data)
df

,Datos,Tiempos_py,Tiempos_numpy,Tiempos_JIT_py,Tiempos_numpy_JIT
0,50,0.000032,0.000049,0.000008,0.000004
1,100,0.000035,0.000020,0.000003,0.000003
2,150,0.000046,0.000017,0.000003,0.000007
3,200,0.000109,0.000042,0.000006,0.000004
4,250,0.000072,0.000020,0.000004,0.000003
5,300,0.000087,0.000018,0.000005,0.000003
6,350,0.000099,0.000016,0.000003,0.000003
7,400,0.000112,0.000015,0.000003,0.000003
8,450,0.000124,0.000016,0.000003,0.000003
9,500,0.000140,0.000016,0.000003,0.000003


In [49]:
px.line(df, y=['Tiempos_py', 'Tiempos_numpy', 'Tiempos_JIT_py', 'Tiempos_numpy_JIT'], x='Datos', log_y=True)

Es posible observar diferencias? A qué se debe esto?

`
Dentro de los gráficos es claro notar las diferencias que existen en 3 partes principales, primero los tiempos de ejecución de la función sin numpy la cual naturalmente es la que más tarda (utiliza funciones iterativas como el for), luego está la función con numpy que tarda menos que la primera al ahorrarse los ciclos iterativos del for mediante el uso de la librería, y finalmente se encuentran las funciones compiladas con Numba el cual es un compilador diseñado para optimizar tareas matemáticas y con ciclos, por lo cual es esperado que estas funciones compilen en un timepo mucho menor al resto.
`

## 1.5 Plot de Resultados de Análisis de Votos [1 Punto]

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [54]:
votos = votes[len(votes) - 1]
posteo = post[len(post) - 1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {
    'Votos (+)': votos[order[:20], 0],
    'Votos (-)': votos[order[:20], 1],
    'Post': np.array(posteo)[order[:20]],
    'url': np.array(url[len(url) - 1])[order[:20]]
}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



In [52]:
r_order = order[::-1][-20:]
data_dic = {
    'mean': posterior_mean[r_order],
    'std_err': std_err[r_order],
    'post': ordered_post[::-1]
}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post", error_x="std_err")
fig.show()

TypeError: 'float' object is not subscriptable

**Respuesta:**

`Escriba su respuesta aquí`

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>